In [1]:
import sys
sys.path.append("../learning/")

import marisa_trie
from dataset import *
from wikidata_linker_utils.offset_array import OffsetArray
import train_type as tp

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [9]:

def get_prob(tagger_ins,sentence_splits):
    ps = tagger_ins.predict_proba_sentences([sentence_splits])
    output = [i for i in ps]
    probs = output[0]['type']
    return probs[0]

language_path = "../data/ja_trie/"

trie_index2indices_values = OffsetArray.load(
    join(language_path, "trie_index2indices")
)
trie_index2indices_counts = OffsetArray(
    np.load(join(language_path, "trie_index2indices_counts.npy")),
    trie_index2indices_values.offsets
)

trie = marisa_trie.Trie().load(
    join(language_path, "trie.marisa")
)

mention = "ジャガー"

min_prob = 0.01
anchor = trie.get(mention)
indices = trie_index2indices_values[anchor]
link_probs = trie_index2indices_counts[anchor]
link_probs = link_probs / link_probs.sum()
mask = link_probs > min_prob
indices = indices[mask]
link_probs = link_probs[mask]

alpha_type_belief = 0.5
beta = 0.99
tagger = tp.SequenceTagger('../ja_model/')
sentence = "ジャガー が ジャングル で ハンティングして いる"
sent_splits = sentence.split()
model_probs = get_prob(tagger,sent_splits)
token_location = sent_splits.index(mention)
type_belief = model_probs[token_location]

type_oracle = load_oracle_classification("../data/classifications/type_classification")
assignments = type_oracle.classify(indices)
type_probs = type_belief[assignments]
type_probs = alpha_type_belief * type_probs + (1.0 - alpha_type_belief)

full_score = link_probs * (1.0 - beta + beta * type_probs)

index = full_score.argmax()
top_pick = indices[index]

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:22: DeprecationWarning: Trie.save is deprecated and will be removed in marisa_trie 0.8.0. Please use Trie.load instead.


INFO:tensorflow:Restoring parameters from ../ja_model/model.ckpt


InternalError: Dst tensor is not initialized.
	 [[Node: save/RestoreV2_18/_31 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device_incarnation=1, tensor_name="edge_106_save/RestoreV2_18", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:GPU:0"]()]]

In [3]:
top_pick

22728920

In [3]:
t2w = marisa_trie.Trie().load(
    "../data/wikidata/wikititle2wikidata.marisa"
)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: Trie.save is deprecated and will be removed in marisa_trie 0.8.0. Please use Trie.load instead.
  


In [1]:
from collections import defaultdict
from tqdm import tqdm_notebook
out = defaultdict(dict)
with open("../data/wikidata/wikidata_wikititle2wikidata.tsv") as f:
    for line in tqdm_notebook(f):
        it = line.replace('\n', '').split('\t')
        target = 'wiki/'
        start = it[0].index(target)
        end = start + len(target)
        k, v = it[0][:start], it[0][end:]
        if(k in ['en', 'ja']):
            out[int(it[1])][k] = v

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

In [2]:
out[0]

{'en': 'Northern Ireland', 'ja': '北アイルランド'}

In [3]:
import pickle
with open("../data/wikidata/indices2title.pkl", 'wb') as f:
    pickle.dump(out, f)

In [6]:
import pickle
with open("../data/wikidata/indices2title.pkl", 'rb') as f:
    out = pickle.load(f)

In [7]:
for index in indices:
    print(out[index]['ja'])

ミック・ジャガー
ジャガー (駆逐艦)
フェンダー・ジャガー
ジャガー (自動車)
ジャガー
ジャガー・レーシング
ジャガー (ローカルタレント)
ジャガー (西城秀樹の曲)


In [91]:
indices

In [8]:
out[top_pick]

{'ar': 'جاغوار (شركة)',
 'ast': 'Xaguar Cars',
 'az': 'Jaguar (avtomobil)',
 'be_x_old': 'Jaguar Cars',
 'bg': 'Ягуар (кола)',
 'bs': 'Jaguar (automobil)',
 'ca': 'Jaguar Cars',
 'ckb': 'جاگوار (ئۆتۆمبێل)',
 'commons': 'Jaguar (automobile)',
 'cs': 'Jaguar',
 'da': 'Jaguar Cars',
 'de': 'Jaguar Cars',
 'en': 'Jaguar Cars',
 'eo': 'Jaguar',
 'es': 'Jaguar Cars',
 'eu': 'Jaguar Cars',
 'fa': 'جگوار کارز',
 'fi': 'Jaguar',
 'fr': 'Jaguar (automobile)',
 'gl': 'Jaguar Cars',
 'he': 'יגואר (יצרן רכב)',
 'hi': 'जैगुआर कारें',
 'hr': 'Jaguar (automobil)',
 'hu': 'Jaguar Cars',
 'hy': 'Jaguar',
 'id': 'Jaguar (perusahaan otomotif)',
 'it': 'Jaguar',
 'ja': 'ジャガー (自動車)',
 'kk': 'Jaguar',
 'kn': 'ಜಗ್ವಾರ್ ಕಾರ್ ಗಳು',
 'ko': '재규어 자동차',
 'ku': 'Jaguar Cars',
 'ln': 'Jaguar',
 'lt': 'Jaguar Cars',
 'lv': 'Jaguar Cars',
 'ml': 'ജാഗ്വാർ കാറുകൾ',
 'mr': 'जॅग्वार कार्स',
 'nah': 'Jaguar',
 'nl': 'Jaguar Cars',
 'no': 'Jaguar (bil)',
 'pl': 'Jaguar (motoryzacja)',
 'pt': 'Jaguar Cars',
 'ro': 'Jaguar Cars

In [72]:
from operator import itemgetter 
ts = ['BC', 'DE', 'EF', 'EEF', 'EFE']
ss = ['A','B','C','D','E','E','F','E']
con = [[] for t in ts]
fin = [False for t in ts]
sep = ''
for i, s in enumerate(ss):
    for j, t in enumerate(ts):
        if fin[j] is True:
            continue            
        if con[j]:
            tmp = list(itemgetter(*con[j])(ss))
            tmp = tmp + [ss[i]]
            ndl = sep.join(tmp)                
        else:
            ndl = ss[i]
        if ndl == t:
            con[j].append(i)
            fin[j] = True
        elif t.startswith(ss[i]) and not t.startswith(ndl):
            con[j] = [i]
            if ss[i] == t:
                fin[j] = True
        elif t.startswith(ndl):
            con[j].append(i)
        else:
            con[j] = []

print(con)

[[1, 2], [3, 4], [5, 6], [4, 5, 6], [5, 6, 7]]


In [71]:
import numpy as np

con_s = con[:]
stack_list = []

def is_all_none(arr):
    for a in arr:
        if a is not None:
            return False
    return True

while(not is_all_none(con_s)):
    appeared_value = []
    appeared_index = []
    stack = []
    for i, c in enumerate(con_s):
        if c is None:
            continue
        if np.sum(np.in1d(c, appeared_value)) > 0:
            continue
        else:
            stack.append(c)
            appeared_index.append(i)
            for v in c:
                appeared_value.append(v)
    for index in appeared_index:
        con_s[index] = None
    stack_list.append(stack)
    
print(stack_list)

[[[1, 2], [3, 4], [5, 6]], [[4, 5, 6]], [[5, 6, 7]]]


In [81]:
the_line = "enwiki/Category:WikiProject Intertranswiki/Croatian"
target = 'wiki/'
start = the_line.index(target)
end = start + len(target)

the_line[:start], the_line[end:]


('en', 'Category:WikiProject Intertranswiki/Croatian')

In [119]:
import marisa_trie
from os.path import join 
language_path = "../data/en_trie/"

trie = marisa_trie.Trie().load(
    join(language_path, "trie.marisa")
)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: Trie.save is deprecated and will be removed in marisa_trie 0.8.0. Please use Trie.load instead.
  


In [126]:
trie.get("Franz Fischler".lower())

8831347

In [2]:
language_path = "../data/ja_trie/"

trie = marisa_trie.Trie().load(
    join(language_path, "trie.marisa")
)


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: Trie.save is deprecated and will be removed in marisa_trie 0.8.0. Please use Trie.load instead.
  after removing the cwd from sys.path.


In [13]:
trie.get('アメリカ中央情報局') is not None

True

In [11]:
assert trie.get('CIA') is not None

AssertionError: 